In [1]:
from pathlib import Path
from IPython.display import HTML, display
css = Path("../../../css/custom.css").read_text(encoding="utf-8")
display(HTML(f"<style>{css}</style>"))

# Chapter 3 — Exploratory Data Analysis (EDA)
## Lesson 9: Leakage Forensics in EDA (Suspicious Features, Post-Outcome Variables)

**What you will learn in this lesson**

- Define data leakage precisely and distinguish it from “legitimate signal.”
- Use EDA to surface leakage *before* modeling decisions become expensive to unwind.
- Detect suspicious features using quick, repeatable diagnostics (single-feature screens, leakage scores, split-sensitivity tests).
- Recognize post-outcome variables and enforce an “as-of time” contract for features.
- Validate suspicion with controlled experiments: swap splits, remove columns, and confirm performance drops are consistent with leakage.
- Establish a reusable leakage-forensics checklist that you can apply to any new dataset.

**Datasets used in this lesson (from the repository)**

We will use multiple datasets to practice leakage forensics across different contexts:

- Classification: `../../../Datasets/Classification/diabetes.csv`
- Regression: `../../../Datasets/Regression/house-prices.csv`
- Real-world tabular with temporal/process columns: `../../../Datasets/Regression/listings.csv`
- Process/workflow dataset (complaints): `../../../Datasets/Clustering/ConsumerComplaints.csv`

## 1) Leakage: a precise definition that is actually useful

In machine learning practice, **leakage** means that your training data (features, preprocessing, or evaluation protocol) contains information that would **not** be available at the moment you intend to make a prediction in the real world.

A practical way to formalize this is to define an **availability boundary**:

- Let $t_0$ be the “prediction time” (the time at which you want to score a new example).
- A feature $x_j$ is valid only if it can be computed using information available at or before $t_0$.
- Leakage occurs when any part of $x$ (or the pipeline that produces $x$) uses information from **after** $t_0$, or otherwise from outside the allowed causal/process boundary.

If you train a model $\hat{y} = f(x)$, leakage can be understood as a violation of the contract:
$$x = g(\text{information available at } t \le t_0).$$

**Why EDA matters:** EDA is where you learn what the columns *really* mean. Many leakage failures happen because the dataset has columns that look “reasonable” but are actually derived from outcomes, operational decisions, or post-hoc measurements.

### Leakage vs. “strong signal”
A feature can legitimately be highly predictive. So “high correlation” alone is not proof. The difference is about *availability* and *process*:

- **Legitimate signal:** available at $t_0$ and causally/process-wise upstream of the outcome.
- **Leakage signal:** created after the outcome (or after key decisions), or directly encodes the label.

In this lesson, you will learn to combine **semantic checks** (column meaning) and **statistical checks** (suspicious predictive strength + split sensitivity) to decide which case you are dealing with.

## 2) A leakage taxonomy you can audit

Leakage shows up in several recurring forms. During EDA, you should actively search for these patterns:

1. **Target leakage (label leakage):** a feature is a direct function of the target (or a near-proxy).  
   Examples: `is_fraud_reviewed`, `chargeback_flag`, `final_grade`, `approved_amount` when predicting approval.

2. **Post-outcome variables:** features recorded after the outcome is known, often as part of a workflow.  
   Examples: `resolution_code`, `time_to_close`, `refund_amount`, `response_status`.

3. **Temporal leakage:** the training split uses future information relative to the test split (time series, cohorts).  
   Example: mixing 2025 data into training for predicting 2024 outcomes.

4. **Group/entity leakage:** the same entity appears in both train and test, leaking identity-specific information.  
   Examples: the same user, patient, device, host, store, or student appearing in both sets.

5. **Duplicate leakage / near-duplicates:** exact or nearly identical rows cross the split boundary.

6. **Preprocessing leakage:** fitting scalers, imputers, encoders, feature selection, or PCA on the full dataset before splitting.

7. **Evaluation leakage:** repeated tuning on the test set, or using the test set to choose features/hyperparameters.

**In EDA, you cannot “prove” leakage with a single plot.** Instead, you build a case using:
- column semantics and process knowledge, plus
- quantitative “suspicion signals,” plus
- controlled experiments that isolate the effect of a suspected leak.

## 3) Toolkit: fast, repeatable diagnostics (code you can reuse)

We will build a small set of helper functions:

- `load_csv(path)`: robust loading (types, missing values).
- `summarize(df)`: EDA summary (types, missingness, cardinality).
- `single_feature_screen(...)`: cross-validated performance using one feature at a time.
- `split_sensitivity_test(...)`: compare performance under different splitting strategies (random vs group vs time).
- `suspicious_name_scan(...)`: catch columns that look like post-outcome proxies by naming patterns.

The goal is not to be perfect; the goal is to quickly identify *which columns deserve investigation*.

In [4]:
import os
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold, TimeSeriesSplit, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.base import clone

import matplotlib.pyplot as plt

np.random.seed(42)

def load_csv(path, **kwargs):
    """Load CSV using a repository-relative path."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}\nCheck the repository structure and your working directory.")
    # Try a couple of safe defaults for messy CSVs.
    defaults = dict(low_memory=False)
    defaults.update(kwargs)
    return pd.read_csv(path, **defaults)

def summarize(df, max_unique=20):
    """Basic EDA summary focused on leakage forensics."""
    out = []
    for c in df.columns:
        s = df[c]
        n = len(s)
        n_missing = int(s.isna().sum())
        miss_rate = n_missing / max(n, 1)
        nunique = int(s.nunique(dropna=True))
        dtype = str(s.dtype)
        example = s.dropna().iloc[0] if s.dropna().shape[0] else np.nan
        out.append({
            "column": c,
            "dtype": dtype,
            "missing_rate": round(miss_rate, 4),
            "nunique": nunique,
            "example_value": example if isinstance(example, (int, float, str)) else str(example),
            "low_cardinality": nunique <= max_unique
        })
    return pd.DataFrame(out).sort_values(["missing_rate", "nunique"], ascending=[False, False]).reset_index(drop=True)

def _infer_task(y):
    # Heuristic: small number of unique values -> classification
    n_unique = y.nunique(dropna=True)
    if n_unique <= 20 and (y.dtype == "object" or y.dtype.name.startswith("int") or y.dtype.name.startswith("bool")):
        return "classification"
    return "regression"

def _build_preprocessor(X):
    # Drop columns that are entirely missing (common in public CSVs and avoids imputer warnings)
    non_empty_cols = [c for c in X.columns if not X[c].isna().all()]
    if len(non_empty_cols) == 0:
        raise ValueError("All feature columns are entirely missing; cannot build a usable preprocessing pipeline.")
    Xn = X[non_empty_cols]

    num_cols = Xn.select_dtypes(include=["number"]).columns.tolist()
    cat_cols = [c for c in Xn.columns if c not in num_cols]

    numeric = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ])
    categorical = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", min_frequency=5)),
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric, num_cols),
            ("cat", categorical, cat_cols),
        ],
        remainder="drop",
        sparse_threshold=0.3
    )
    return pre, num_cols, cat_cols

def single_feature_screen(df, target_col, task=None, max_cat_unique=50, cv_splits=5, random_state=42):
    """Cross-validated performance using one feature at a time. High scores can indicate leakage."""
    y = df[target_col]
    X_all = df.drop(columns=[target_col]).copy()

    if task is None:
        task = _infer_task(y)

    results = []
    # Ignore very high-cardinality text/id-like columns to keep the screen fast.
    for col in X_all.columns:
        s = X_all[col]
        nunique = s.nunique(dropna=True)
        if s.dtype == "object" and nunique > max_cat_unique:
            continue

        X = X_all[[col]].copy()

        pre, _, _ = _build_preprocessor(X)

        if task == "classification":
            # Use a robust linear baseline
            model = LogisticRegression(max_iter=2000, solver="lbfgs")
            pipe = Pipeline([("pre", pre), ("model", model)])
            cv = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=random_state)
            scores = cross_val_score(pipe, X, y, cv=cv, scoring="roc_auc")
            metric_name = "roc_auc"
        else:
            model = Ridge(alpha=1.0, random_state=random_state)
            pipe = Pipeline([("pre", pre), ("model", model)])
            cv = KFold(n_splits=cv_splits, shuffle=True, random_state=random_state)
            scores = cross_val_score(pipe, X, y, cv=cv, scoring="r2")
            metric_name = "r2"

        results.append({
            "feature": col,
            metric_name: float(np.mean(scores)),
            "std": float(np.std(scores)),
            "nunique": int(nunique),
            "dtype": str(s.dtype)
        })

    if not results:
        return pd.DataFrame(columns=["feature", "score", "std", "nunique", "dtype"])

    res = pd.DataFrame(results)
    score_col = "roc_auc" if task == "classification" else "r2"
    return res.sort_values(score_col, ascending=False).reset_index(drop=True)

def split_sensitivity_test(df, target_col, drop_cols=None, group_col=None, time_col=None, task=None, cv_splits=5, random_state=42):
    """Compare performance across splitting strategies. Large deltas can indicate leakage."""
    drop_cols = drop_cols or []
    y = df[target_col]
    X = df.drop(columns=[target_col] + drop_cols).copy()

    if task is None:
        task = _infer_task(y)

    pre, _, _ = _build_preprocessor(X)

    if task == "classification":
        model = LogisticRegression(max_iter=2000, solver="lbfgs")
        scoring = "roc_auc"
    else:
        model = Ridge(alpha=1.0, random_state=random_state)
        scoring = "r2"

    pipe = Pipeline([("pre", pre), ("model", model)])

    scores = {}

    # Random CV
    if task == "classification":
        cv_random = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=random_state)
    else:
        cv_random = KFold(n_splits=cv_splits, shuffle=True, random_state=random_state)
    scores["random_cv"] = float(np.mean(cross_val_score(pipe, X, y, cv=cv_random, scoring=scoring)))

    # Group CV (if provided)
    if group_col is not None:
        groups = df[group_col].values
        cv_group = GroupKFold(n_splits=min(cv_splits, len(np.unique(groups))))
        scores["group_cv"] = float(np.mean(cross_val_score(pipe, X, y, cv=cv_group, groups=groups, scoring=scoring)))

    # Time CV (if provided)
    if time_col is not None:
        # Sort by time and use expanding splits
        order = pd.to_datetime(df[time_col], errors="coerce").sort_values().index
        X_t = X.loc[order]
        y_t = y.loc[order]
        tscv = TimeSeriesSplit(n_splits=cv_splits)
        scores["time_cv"] = float(np.mean(cross_val_score(pipe, X_t, y_t, cv=tscv, scoring=scoring)))

    return scores

def suspicious_name_scan(columns):
    """Flag column names that often correlate with leakage in real projects."""
    patterns = [
        r"label", r"target", r"outcome", r"response", r"resolved", r"resolution", r"closed", r"approved",
        r"final", r"post", r"after", r"future", r"leak", r"score", r"status", r"decision", r"payout",
        r"refund", r"chargeback", r"dispute"
    ]
    rx = re.compile("|".join(patterns), flags=re.IGNORECASE)
    flagged = [c for c in columns if rx.search(c)]
    return flagged

## 4) Case study A — Diabetes classification: “too good to be true” features

We will load the diabetes dataset and run a standard EDA summary. Then we will **simulate** a common leakage failure: a feature that is (directly or indirectly) created from the label.

This simulation is intentional: in real projects, leakage often hides inside columns that appear “reasonable” (e.g., codes, derived flags, post-hoc measurements). By creating a leaky feature ourselves, we can verify that our diagnostics actually catch it.

**Target:** `classification` (Diabetic vs Non-Diabetic)

**Key forensics questions**
- Are any columns suspicious by name?
- Does any single feature achieve extremely high ROC-AUC by itself?
- Does overall performance collapse if the suspicious feature is removed?

### 4.1 Load and summarize

Start by inspecting missingness, data types, and cardinality. Leakage often correlates with:
- columns that are IDs (high cardinality),
- workflow/status fields,
- timestamps or “final outcome” fields,
- post-processed aggregates.

In [5]:
# --- Case study A: diabetes.csv ---
diabetes_path = "../../../Datasets/Classification/diabetes.csv"
df_diabetes = load_csv(diabetes_path)

display(df_diabetes.head())
summary_diabetes = summarize(df_diabetes)
display(summary_diabetes)

# Suspicious name scan
flagged = suspicious_name_scan(df_diabetes.columns)
print("Flagged by name patterns:", flagged)

# Ensure target is binary for ROC-AUC
df_diabetes = df_diabetes.copy()
df_diabetes["y"] = (df_diabetes["classification"].astype(str).str.lower() == "diabetic").astype(int)

# Inject a leaky proxy: label plus tiny noise (mimics a post-outcome proxy)
rng = np.random.default_rng(42)
df_diabetes["leaky_proxy"] = df_diabetes["y"] + rng.normal(0, 0.03, size=len(df_diabetes))

# Build a modeling frame
df_d = df_diabetes.drop(columns=["classification"]).copy()

# Single-feature screen
screen_d = single_feature_screen(df_d, target_col="y", task="classification", max_cat_unique=50, cv_splits=5)
display(screen_d.head(15))

# Split sensitivity with and without the leaky feature
scores_with = split_sensitivity_test(df_d, target_col="y", task="classification", cv_splits=5)
scores_without = split_sensitivity_test(df_d.drop(columns=["leaky_proxy"]), target_col="y", task="classification", cv_splits=5)
print("Scores WITH leaky feature:", scores_with)
print("Scores WITHOUT leaky feature:", scores_without)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,classification
0,6,148,72,35,0,33.6,0.627,50,Diabetic
1,1,85,66,29,0,26.6,0.351,31,Non-Diabetic
2,8,183,64,0,0,23.3,0.672,32,Diabetic
3,1,89,66,23,94,28.1,0.167,21,Non-Diabetic
4,0,137,40,35,168,43.1,2.288,33,Diabetic


,column,dtype,missing_rate,nunique,example_value,low_cardinality
0,DiabetesPedigreeFunction,float64,0.0,517,0.627,False
1,BMI,float64,0.0,248,33.6,False
2,Insulin,int64,0.0,186,0,False
3,Glucose,int64,0.0,136,148,False
4,Age,int64,0.0,52,50,False
5,SkinThickness,int64,0.0,51,35,False
6,BloodPressure,int64,0.0,47,72,False
7,Pregnancies,int64,0.0,17,6,True
8,classification,object,0.0,2,Diabetic,True


Flagged by name patterns: []


,feature,roc_auc,std,nunique,dtype
0,leaky_proxy,1.000000,0.000000,768,float64
1,Glucose,0.787365,0.026682,136,int64
2,BMI,0.688052,0.032472,248,float64
3,Age,0.685324,0.037894,52,int64
4,Pregnancies,0.619985,0.037818,17,int64
5,DiabetesPedigreeFunction,0.603836,0.052310,517,float64
6,BloodPressure,0.587046,0.037250,47,int64
7,SkinThickness,0.553768,0.049009,51,int64
8,Insulin,0.536359,0.070975,186,int64


Scores WITH leaky feature: {'random_cv': 1.0}
Scores WITHOUT leaky feature: {'random_cv': 0.8303878406708595}


### 4.2 Inject a leaky proxy and detect it

We create a synthetic feature `leaky_proxy` that encodes the label with small noise.
In a real dataset, this could correspond to a “post-diagnosis treatment code” or a “billing class” that effectively reveals the target.

Your diagnostics should rank this feature near the top in the single-feature screen.

### 4.3 Interpreting results (how to reason, not just compute)

If you see a feature with **ROC-AUC close to 1.0** in a single-feature screen, treat it as a red alert:

- It might be a direct encoding of the target.
- It might be a deterministic consequence of the target (post-outcome).
- It might be an ID that indirectly reveals the target because of data collection artifacts.

The correct next step is not “drop it immediately,” but to ask:

1. **When is this feature available?** (Define $t_0$.)
2. **Who/what generates it?** (Operational workflow? Human review? Billing?)
3. **Can it be computed without knowing the target?** (If not, it is leakage.)

In real projects, these questions are answered with domain experts, data lineage, and system logs. In EDA, your job is to *surface the candidates*.

## 5) Case study B — House prices regression: leakage via “derived from target” features

In regression, leakage often happens when someone creates “helpful” features that are algebraic rearrangements of the target. For example:

- predicting `Price` and adding a feature `Price_per_sqft = Price / SqFt`  
  This is effectively giving the model the answer.

We will:
- load `house-prices.csv`,
- build a baseline model to estimate performance,
- inject two leaky engineered features,
- confirm that the leakage shows up in the single-feature screen and in performance inflation.

**Target:** `Price`

**Red flag heuristic:** if a feature is *mathematically constrained* by the target, $x_j = h(y, \cdot)$, then it is not a valid feature for predicting $y$ at scoring time.

In [6]:
# --- Case study B: house-prices.csv ---
house_path = "../../../Datasets/Regression/house-prices.csv"
df_house = load_csv(house_path)

display(df_house.head())
display(summarize(df_house))

# Baseline: predict Price
df_h = df_house.copy()

# Inject leaky engineered features
df_h["price_per_sqft_leak"] = df_h["Price"] / df_h["SqFt"].replace(0, np.nan)
df_h["log_price_leak"] = np.log1p(df_h["Price"])

# Screen single features for regression
screen_h = single_feature_screen(df_h, target_col="Price", task="regression", max_cat_unique=50, cv_splits=5)
display(screen_h.head(15))

# Compare performance with and without leaky columns
scores_with = split_sensitivity_test(df_h, target_col="Price", task="regression", cv_splits=5)
scores_without = split_sensitivity_test(df_h.drop(columns=["price_per_sqft_leak", "log_price_leak"]), target_col="Price", task="regression", cv_splits=5)

print("Regression scores WITH leaky features:", scores_with)
print("Regression scores WITHOUT leaky features:", scores_without)

,Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood
0,1,114300,1790,2,2,2,No,East
1,2,114200,2030,4,2,3,No,East
2,3,114800,1740,3,2,1,No,East
3,4,94700,1980,3,2,3,No,East
4,5,119800,2130,3,3,3,No,East


,column,dtype,missing_rate,nunique,example_value,low_cardinality
0,Home,int64,0.0,128,1,False
1,Price,int64,0.0,123,114300,False
2,SqFt,int64,0.0,61,1790,False
3,Offers,int64,0.0,6,2,True
4,Bedrooms,int64,0.0,4,2,True
5,Bathrooms,int64,0.0,3,2,True
6,Neighborhood,object,0.0,3,East,True
7,Brick,object,0.0,2,No,True


,feature,r2,std,nunique,dtype
0,log_price_leak,0.977711,0.007256,123,float64
1,price_per_sqft_leak,0.683828,0.083864,128,float64
2,Neighborhood,0.450231,0.126492,3,object
3,SqFt,0.228876,0.100532,61,int64
4,Bedrooms,0.227696,0.182803,4,int64
5,Bathrooms,0.120441,0.232334,3,int64
6,Brick,0.092262,0.104749,2,object
7,Offers,-0.016599,0.113085,6,int64
8,Home,-0.104937,0.113642,128,int64


Regression scores WITH leaky features: {'random_cv': 0.9895991548830227}
Regression scores WITHOUT leaky features: {'random_cv': 0.8275952075387714}


## 6) Case study C — Listings data: post-outcome columns and split sensitivity

Real-world datasets often contain columns that are only known **after** some operational history has occurred.
For listings data, common post-outcome-ish columns include:
- `number_of_reviews`, `reviews_per_month`, `last_review` (depend on future bookings/reviews)
- `availability_365` (depends on future calendar occupancy)

Suppose your intended prediction is: “Estimate the listing price at the time the listing is created.”
Then review-related columns may violate the $t_0$ contract.

In addition, listings have natural **groups** (e.g., `host_id`). If the same host appears in both train and test, random CV can look overly optimistic.

We will demonstrate:
- name-based suspicion scan,
- group split vs random split,
- time-aware split (when a usable time column exists).

In [7]:
# --- Case study C: listings.csv ---
listings_path = "../../../Datasets/Regression/listings.csv"
df_listings = load_csv(listings_path)

display(df_listings.head())
display(summarize(df_listings))

# Define a simple target to keep the tutorial self-contained:
# Predict whether a listing is "high price" (binary classification).
# (We ignore rows with missing price.)
dfl = df_listings.copy()
dfl["price"] = pd.to_numeric(dfl["price"], errors="coerce")
dfl = dfl.dropna(subset=["price"]).reset_index(drop=True)

# Median-based label (robust and easy to explain)
median_price = float(dfl["price"].median())
dfl["y_high_price"] = (dfl["price"] >= median_price).astype(int)

# Columns that may be post-outcome relative to listing creation time
suspected_post_outcome = ["number_of_reviews", "last_review", "reviews_per_month", "number_of_reviews_ltm", "availability_365"]
present_suspects = [c for c in suspected_post_outcome if c in dfl.columns]
print("Potential post-outcome columns present:", present_suspects)

# Quick name scan
flagged = suspicious_name_scan(dfl.columns)
print("Flagged by name patterns:", flagged)

# Compare random CV vs group CV (host_id)
drop_cols = []
for c in ["id", "name", "host_name", "license"]:
    if c in dfl.columns:
        drop_cols.append(c)

# Include suspects first, then compare when we drop them
scores_random_vs_group = split_sensitivity_test(
    dfl.drop(columns=drop_cols),
    target_col="y_high_price",
    task="classification",
    group_col="host_id" if "host_id" in dfl.columns else None,
    time_col="last_review" if "last_review" in dfl.columns else None,
    cv_splits=5
)
print("Scores (random/group/time if available):", scores_random_vs_group)

# Now drop suspected post-outcome features and re-test
cols_to_drop = [c for c in present_suspects if c in dfl.columns]
dfl_noleak = dfl.drop(columns=cols_to_drop + drop_cols, errors="ignore")
scores_no_post = split_sensitivity_test(
    dfl_noleak,
    target_col="y_high_price",
    task="classification",
    group_col="host_id" if "host_id" in dfl_noleak.columns else None,
    time_col="last_review" if "last_review" in dfl_noleak.columns else None,
    cv_splits=5
)
print("Scores after dropping suspected post-outcome columns:", scores_no_post)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,13913,Holiday London DB Room Let-on going,54730,Alina,NaN,Islington,51.56861,-0.11270,Private room,57.0,1,51,2025-02-09,0.29,3,344,10,NaN
1,15400,Bright Chelsea Apartment. Chelsea!,60302,Philippa,NaN,Kensington and Chelsea,51.48780,-0.16813,Entire home/apt,NaN,4,96,2024-04-28,0.52,1,11,2,NaN
2,17402,Very Central Modern 3-Bed/2 Bath By Oxford St W1,67564,Liz,NaN,Westminster,51.52195,-0.14094,Entire home/apt,510.0,3,56,2024-02-19,0.33,5,293,0,NaN
3,24328,Battersea live/work artist house,41759,Joe,NaN,Wandsworth,51.47072,-0.16266,Entire home/apt,213.0,90,94,2022-07-19,0.54,1,194,0,NaN
4,31036,Bright compact 1 Bedroom Apartment Brick Lane,133271,Hendryks,NaN,Tower Hamlets,51.52425,-0.06997,Entire home/apt,100.0,2,126,2025-02-20,0.70,8,353,3,NaN


,column,dtype,missing_rate,nunique,example_value,low_cardinality
0,neighbourhood_group,float64,1.0000,0,NaN,True
1,license,float64,1.0000,0,NaN,True
2,price,float64,0.3619,1239,57.0,False
3,last_review,object,0.2564,3444,2025-02-09,False
4,reviews_per_month,float64,0.2564,868,0.29,False
5,host_name,object,0.0006,16416,Alina,False
6,id,int64,0.0000,94559,13913,False
7,name,object,0.0000,90942,Holiday London DB Room Let-on going,False
8,longitude,float64,0.0000,61792,-0.1127,False
9,host_id,int64,0.0000,55395,54730,False


Potential post-outcome columns present: ['number_of_reviews', 'last_review', 'reviews_per_month', 'number_of_reviews_ltm', 'availability_365']
Flagged by name patterns: []
Scores (random/group/time if available): {'random_cv': 0.9994904216839016, 'group_cv': 0.9994761209249441, 'time_cv': 0.9973845805463218}
Scores after dropping suspected post-outcome columns: {'random_cv': 0.9997040883162702, 'group_cv': 0.9996974451218641}


### 6.1 How to interpret split sensitivity

When **random CV** is much higher than **group CV**, a typical explanation is entity leakage:

- The model learns host-specific patterns (or neighborhood identity proxies).
- The test fold contains the same hosts, making the problem easier than real deployment.

When **time-aware CV** is worse than random CV, a typical explanation is temporal leakage and distribution shift:

- future patterns leak into training when you ignore time,
- the data-generating process changes across time (concept drift).

Split sensitivity does not *prove* leakage. It tells you **where to investigate**:
- Which columns are effectively “ID-like”?
- Which columns require an as-of-time policy?
- Are you accidentally learning a cohort artifact rather than a generalizable relationship?

## 7) Case study D — Consumer complaints: post-outcome workflow columns

The complaints dataset is an excellent leakage playground because it contains both:
- early-stage intake information (what is known when a complaint arrives), and
- later-stage workflow outcomes (what happens after the complaint is processed).

To make the “as-of time” boundary explicit, define a realistic prediction:

**Prediction objective:** predict whether a consumer will dispute the resolution (`Consumer Disputed`) *at the time the complaint is received*.

At that moment, many columns are **not** available yet:
- `Company Response to Consumer`
- `Timely Response`
- `Date Sent to Company`
- `Company Public Response`
- sometimes even internal status codes

Using these columns can create severe leakage: you are using the resolution process to predict whether the consumer disputes it.

We will:
- load the dataset,
- build two feature sets (intake-only vs intake+workflow),
- compare performance,
- and identify the columns that create the biggest inflation.

In [8]:
# --- Case study D: ConsumerComplaints.csv ---
complaints_path = "../../../Datasets/Clustering/ConsumerComplaints.csv"
df_cc = load_csv(complaints_path)

display(df_cc.head())
display(summarize(df_cc))

# Clean up target: Consumer Disputed (Yes/No)
dfc = df_cc.copy()
if "Consumer Disputed" not in dfc.columns:
    raise ValueError("Expected column 'Consumer Disputed' not found in ConsumerComplaints.csv")

# Drop rows where target is missing
dfc = dfc.dropna(subset=["Consumer Disputed"]).reset_index(drop=True)

dfc["y_disputed"] = (dfc["Consumer Disputed"].astype(str).str.strip().str.lower() == "yes").astype(int)

# Define intake-time columns (best-effort, based on typical intake fields)
intake_cols = []
for c in ["Date Received", "Product Name", "Sub Product", "Issue", "Sub Issue", "Company", "State Name", "Zip Code", "Tags", "Submitted via"]:
    if c in dfc.columns:
        intake_cols.append(c)

# Define workflow/post-outcome-ish columns (likely not known at intake time)
workflow_cols = []
for c in ["Date Sent to Company", "Company Response to Consumer", "Company Public Response", "Timely Response", "Complaint ID"]:
    if c in dfc.columns:
        workflow_cols.append(c)

print("Intake columns:", intake_cols)
print("Workflow columns:", workflow_cols)

# Build two datasets:
# 1) intake-only
df_intake = dfc[intake_cols + ["y_disputed"]].copy()

# 2) intake + workflow
df_full = dfc[intake_cols + workflow_cols + ["y_disputed"]].copy()

# Evaluate split sensitivity (random CV) as a quick proxy
scores_intake = split_sensitivity_test(df_intake, target_col="y_disputed", task="classification", cv_splits=5)
scores_full = split_sensitivity_test(df_full, target_col="y_disputed", task="classification", cv_splits=5)

print("Intake-only scores:", scores_intake)
print("Intake+workflow scores:", scores_full)

# Single-feature screen on the full set to see which columns are suspiciously predictive
screen_cc = single_feature_screen(df_full, target_col="y_disputed", task="classification", max_cat_unique=50, cv_splits=5)
display(screen_cc.head(20))

,Date Received,Product Name,Sub Product,Issue,Sub Issue,Consumer Complaint Narrative,Company Public Response,Company,State Name,Zip Code,Tags,Consumer Consent Provided,Submitted via,Date Sent to Company,Company Response to Consumer,Timely Response,Consumer Disputed,Complaint ID
0,2013-07-29,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,NaN,NaN,Wells Fargo & Company,VA,24540,NaN,NaN,Phone,2013-07-30,Closed with explanation,Yes,No,468882
1,2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,NaN,NaN,NaN,Wells Fargo & Company,CA,95992,Older American,NaN,Web,2013-07-31,Closed with explanation,Yes,No,468889
2,2013-07-29,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,Santander Bank US,NY,10065,NaN,NaN,Fax,2013-07-31,Closed,Yes,No,468879
3,2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,Wells Fargo & Company,GA,30084,NaN,NaN,Web,2013-07-30,Closed with explanation,Yes,No,468949
4,2013-07-29,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Franklin Credit Management,CT,6106,NaN,NaN,Web,2013-07-30,Closed with explanation,Yes,No,475823


,column,dtype,missing_rate,nunique,example_value,low_cardinality
0,Company Public Response,object,0.9626,9,Company chooses not to provide a public response,True
1,Consumer Complaint Narrative,object,0.9601,2592,Received Capital One charge card offer XXXX. A...,False
2,Consumer Consent Provided,object,0.9273,3,Consent provided,True
3,Tags,object,0.8522,3,Older American,True
4,Sub Issue,object,0.5319,67,Debt is not mine,False
5,Sub Product,object,0.2834,45,Vehicle loan,False
6,Consumer Disputed,object,0.0169,2,No,True
7,Zip Code,object,0.0073,14851,24540,False
8,State Name,object,0.0073,59,VA,False
9,Complaint ID,int64,0.0000,65499,468882,False


Intake columns: ['Date Received', 'Product Name', 'Sub Product', 'Issue', 'Sub Issue', 'Company', 'State Name', 'Zip Code', 'Tags', 'Submitted via']
Workflow columns: ['Date Sent to Company', 'Company Response to Consumer', 'Company Public Response', 'Timely Response', 'Complaint ID']
Intake-only scores: {'random_cv': 0.5900515995185385}
Intake+workflow scores: {'random_cv': 0.6054917298073457}


,feature,roc_auc,std,nunique,dtype
0,Company Response to Consumer,0.553540,0.003773,4,object
1,Product Name,0.544824,0.009093,11,object
2,Sub Product,0.543912,0.006271,45,object
3,Submitted via,0.543063,0.001527,6,object
4,Complaint ID,0.505308,0.004983,64391,int64
5,Company Public Response,0.501944,0.000621,9,object
6,Tags,0.501036,0.001619,3,object
7,Timely Response,0.500367,0.001001,2,object


### 7.1 “As-of time” thinking: the discipline that prevents leakage

Leakage prevention becomes tractable when you explicitly document an **as-of time contract**:

- What is $t_0$ for this prediction?
- Which source systems are allowed?
- Which events must have occurred before a feature can exist?
- Which joins/aggregations are permitted?

In a real feature store, this is enforced with event-time joins and point-in-time correctness.
In notebooks, you enforce it by:
- writing down the availability assumption,
- restricting the feature set to columns that satisfy it,
- and validating via time/group splits.

A useful heuristic:

- If a feature is generated by a downstream process that is triggered by the target (or by actions taken after observing the target), it is likely post-outcome and invalid at $t_0$.

This is why columns like “resolution,” “closed,” “timely response,” and “final status” are frequent leakage culprits.

## 8) Beyond the basics: stronger leakage forensics patterns

The quick screens above are intentionally lightweight. In higher-stakes work, you should add deeper checks.

### 8.1 Conditional performance inflation test

The defining symptom of leakage is that the model performs well because it can “peek” at restricted information.
A practical test is:

1. Train a baseline on the “safe” feature set.
2. Add one suspected column.
3. Re-train under the same evaluation protocol.
4. If performance jumps sharply and the column is not available at $t_0$, treat it as leakage.

### 8.2 Robustness to split strategy

Compare:
- random K-fold,
- group K-fold (entity-aware),
- time-aware splits (when applicable).

If a feature is truly generalizable, performance should not collapse catastrophically under a more realistic split.
A collapse does not automatically mean leakage (distribution shift can do this), but it is a strong investigation signal.

### 8.3 Duplicate / near-duplicate detection

In EDA, check for:
- exact duplicate rows,
- duplicates on key identifiers,
- near-duplicates (high similarity across many fields).

Leakage via duplicates is common in scraped or merged datasets.

### 8.4 Preprocessing leakage audit

If you compute any statistic using the full dataset (mean, variance, PCA, target encoding, feature selection), you risk leakage.
The safe pattern is always:

- split first,
- fit preprocessing on train only,
- apply to validation/test via `transform`.

In scikit-learn, this is naturally enforced when you put preprocessing inside a `Pipeline` and cross-validate the pipeline.

## 8.5 Target encoding and aggregation leakage (a common EDA trap)

Some of the most subtle leakage bugs are introduced *while doing EDA*—especially when you build “helpful summaries” of the target.

### Target encoding: why it leaks so easily

Target encoding replaces a category with a statistic computed from the target, commonly:

$$\mathrm{TE}(c) = \mathbb{E}[y \mid x=c].$$

If you compute $\mathrm{TE}(c)$ using the full dataset and then cross-validate, every fold’s validation rows have influenced the encoding. This is leakage through the preprocessing step.

The correct approach is **cross-fitting**:
- for each fold, compute encodings using only that fold’s training portion,
- apply them to that fold’s validation portion.

Below, we will demonstrate the performance inflation from naïve target encoding versus cross-fitted target encoding on the diabetes dataset.

Even if you never plan to deploy target encoding, this example is valuable because it teaches a general lesson:

- Any feature derived from the target must be computed in a way that respects the split boundary.

### 8.6 Point-in-time correctness for time-based aggregates

Another frequent leakage pattern is building aggregates that unknowingly include future information. Consider a setting with events $(e_i, t_i)$:

- You want to predict an outcome at $t_0$.
- You compute an aggregate like “customer total spend” by summing transactions.
- If the sum includes transactions with $t > t_0$, your feature violates the availability contract.

In symbols, a safe aggregate is:
$$\mathrm{agg}(t_0) = \sum_{i: t_i \le t_0} v_i,$$
not
$$\sum_{i} v_i.$$

In tabular datasets without explicit event logs, this can still happen when:
- you compute averages per group using the full dataset,
- you compute “historical rates” using future rows,
- you compute rolling statistics but do not respect chronological order.

In the listings dataset, we will create a deliberately leaky group aggregate that uses the *label* to compute a host-level statistic, then show how to compute a safer variant using cross-fitting.

In [9]:
# --- Demonstration: naïve target encoding vs cross-fitted target encoding ---

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

def naive_target_encode(series, y):
    """Compute target mean per category using the full dataset (THIS LEAKS under CV)."""
    df_tmp = pd.DataFrame({"x": series.astype(str), "y": y})
    means = df_tmp.groupby("x")["y"].mean()
    return series.astype(str).map(means).astype(float)

def cv_target_encode(series, y, cv):
    """Cross-fitted target encoding: encodings computed on train folds only."""
    series = series.astype(str)
    y = np.asarray(y)
    enc = np.zeros(len(series), dtype=float)

    for tr_idx, va_idx in cv.split(series, y):
        x_tr = series.iloc[tr_idx]
        y_tr = y[tr_idx]
        means = pd.DataFrame({"x": x_tr, "y": y_tr}).groupby("x")["y"].mean()
        # For unseen categories in validation, use the global mean of the training fold
        global_mean = float(np.mean(y_tr))
        enc_va = series.iloc[va_idx].map(means).fillna(global_mean).astype(float)
        enc[va_idx] = enc_va.values

    return enc

# Use diabetes dataset with a simple categorical binning of Age
d = df_diabetes.copy()
d["age_bin"] = pd.cut(d["Age"], bins=[0, 25, 35, 45, 55, 100], include_lowest=True).astype(str)

y = d["y"].values
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Naïve target encoding (leaky under CV)
te_naive = naive_target_encode(d["age_bin"], y)

# Cross-fitted target encoding (respects CV boundary)
te_cv = cv_target_encode(d["age_bin"], y, cv)

# Evaluate AUC using the encoded value as a score (no model needed)
auc_naive = []
auc_cv = []
for tr_idx, va_idx in cv.split(d, y):
    auc_naive.append(roc_auc_score(y[va_idx], te_naive.iloc[va_idx]))
    auc_cv.append(roc_auc_score(y[va_idx], te_cv[va_idx]))

print("Naïve TE mean AUC (leaky):", float(np.mean(auc_naive)))
print("Cross-fitted TE mean AUC:", float(np.mean(auc_cv)))

# --- Demonstration: leaky group aggregate on listings ---

if "host_id" in dfl.columns:
    temp = dfl.copy()

    # Deliberately leaky: uses the label y_high_price to compute host-level rate using ALL rows
    host_rate_leaky = temp.groupby("host_id")["y_high_price"].mean()
    temp["host_highprice_rate_leaky"] = temp["host_id"].map(host_rate_leaky)

    # Cross-fitted host rate: compute host rates on train fold only
    def cv_group_rate(df, group_col, y_col, cv):
        y = df[y_col].values
        out = np.zeros(len(df), dtype=float)
        for tr_idx, va_idx in cv.split(df, y):
            tr = df.iloc[tr_idx]
            va = df.iloc[va_idx]
            rates = tr.groupby(group_col)[y_col].mean()
            global_mean = float(tr[y_col].mean())
            out[va_idx] = va[group_col].map(rates).fillna(global_mean).astype(float).values
        return out

    cv2 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    temp["host_highprice_rate_cv"] = cv_group_rate(temp, "host_id", "y_high_price", cv2)

    # Evaluate how predictive these two versions are (AUC as a proxy)
    auc_leaky = []
    auc_safe = []
    y2 = temp["y_high_price"].values
    for tr_idx, va_idx in cv2.split(temp, y2):
        auc_leaky.append(roc_auc_score(y2[va_idx], temp.loc[va_idx, "host_highprice_rate_leaky"]))
        auc_safe.append(roc_auc_score(y2[va_idx], temp.loc[va_idx, "host_highprice_rate_cv"]))

    print("Leaky host-rate mean AUC:", float(np.mean(auc_leaky)))
    print("Cross-fitted host-rate mean AUC:", float(np.mean(auc_safe)))
else:
    print("host_id column not found in listings sample; skipping host-rate demo.")

Naïve TE mean AUC (leaky): 0.6847631027253668
Cross-fitted TE mean AUC: 0.6815408805031445
Leaky host-rate mean AUC: 0.9843660815848916
Cross-fitted host-rate mean AUC: 0.7843591638684562


In [10]:
# --- Duplicate and near-duplicate checks (generic utilities) ---

def duplicate_report(df, subset=None):
    if subset is None:
        dup = df.duplicated(keep=False)
    else:
        dup = df.duplicated(subset=subset, keep=False)
    n_dup = int(dup.sum())
    print(f"Duplicate rows (subset={subset}): {n_dup}")
    if n_dup > 0:
        display(df.loc[dup].head(20))

# Examples: run on each dataset
duplicate_report(df_diabetes)
duplicate_report(df_house)
duplicate_report(df_listings, subset=["host_id"] if "host_id" in df_listings.columns else None)

Duplicate rows (subset=None): 0
Duplicate rows (subset=None): 0
Duplicate rows (subset=['host_id']): 49863


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,13913,Holiday London DB Room Let-on going,54730,Alina,NaN,Islington,51.56861,-0.112700,Private room,57.0,1,51,2025-02-09,0.29,3,344,10,NaN
2,17402,Very Central Modern 3-Bed/2 Bath By Oxford St W1,67564,Liz,NaN,Westminster,51.52195,-0.140940,Entire home/apt,510.0,3,56,2024-02-19,0.33,5,293,0,NaN
4,31036,Bright compact 1 Bedroom Apartment Brick Lane,133271,Hendryks,NaN,Tower Hamlets,51.52425,-0.069970,Entire home/apt,100.0,2,126,2025-02-20,0.70,8,353,3,NaN
5,33332,Beautiful Ensuite Richmond-upon-Thames borough,144444,Chi-Chi,NaN,Richmond upon Thames,51.46410,-0.324980,Private room,133.0,2,19,2022-08-01,0.11,2,365,0,NaN
7,36660,You are GUARANTEED to love this,157884,Agri & Roger,NaN,Haringey,51.58478,-0.160570,Private room,74.0,2,711,2025-02-26,4.03,2,193,51,NaN
8,38610,CHARMING FAMILY HOME,165579,Elisa & Dom,NaN,Hammersmith and Fulham,51.50701,-0.233620,Entire home/apt,NaN,91,42,2023-08-27,0.27,2,76,0,NaN
9,38950,Room 1 Large Double Bedroom - front ground floor,167107,Paul,NaN,Haringey,51.58684,-0.086320,Private room,52.0,1,1,2021-12-12,0.03,2,89,0,NaN
14,41712,"Room with a view, shared flat, central Bankside",182322,Nina,NaN,Southwark,51.50191,-0.101998,Private room,NaN,2,131,2024-10-14,0.77,2,179,7,NaN
16,42010,You Will Save Money Here,157884,Agri & Roger,NaN,Barnet,51.58590,-0.164340,Private room,55.0,2,613,2025-03-13,3.48,2,175,50,NaN
18,43129,Quiet Comfortable Room in Fulham,188138,Sylvan,NaN,Hammersmith and Fulham,51.48164,-0.210820,Private room,66.0,3,248,2025-01-02,1.75,3,242,9,NaN


## 9) A practical leakage-forensics checklist (use this every time)

Use this checklist during EDA, before you commit to modeling:

**A. Define the prediction contract**
- What exactly is $y$?
- What is $t_0$ (prediction time)?
- Which sources are available at $t_0$?

**B. Column semantics**
- Identify workflow/status fields, “final” outcomes, and post-hoc summaries.
- Identify IDs and keys (high cardinality).
- Identify timestamp fields and the events they represent.

**C. Quick suspicion screens**
- Name-pattern scan (`suspicious_name_scan`).
- Single-feature performance screen (`single_feature_screen`).

**D. Controlled experiments**
- Remove suspected columns and confirm performance drops.
- Compare random vs group vs time splits (`split_sensitivity_test`).

**E. Preprocessing hygiene**
- Ensure all transforms are inside a pipeline.
- Never fit preprocessing on the full dataset before splitting.

**F. Documentation**
- Record which columns were excluded and why.
- Record the as-of contract assumptions.

If you make this a habit, leakage becomes something you catch early instead of a catastrophe you debug at the end.

## 10) Exercises

1. **Diabetes:** create a second leaky feature that mimics a “post-diagnosis indicator” (e.g., a binary flag derived from the target) and confirm it is detected.
2. **House prices:** create a “leaky bucket” feature: `PriceBucket = round(Price / 10000)` and measure the performance inflation.
3. **Listings:** rerun the split sensitivity test after removing `host_id` and any review-related columns. How much does group CV change?
4. **Complaints:** redefine $t_0$ to be “after the complaint was sent to the company” and decide which workflow columns become valid. How do results change?

When you can answer these questions, you have moved from “knowing leakage exists” to being able to *forensically prove it* and prevent it.

## 11) Summary

- Leakage is a violation of an availability boundary: features (or preprocessing/evaluation) include restricted information.
- EDA is the ideal phase to detect leakage because column meaning and data lineage issues surface here.
- Combine semantic checks with quantitative screens:
  - suspicious names,
  - unusually high single-feature performance,
  - split sensitivity (random vs group vs time),
  - controlled “add/remove a column” experiments.
- Make an as-of-time contract explicit. Post-outcome workflow columns are the most common leakage source in real data.
- Enforce preprocessing hygiene with pipelines and train-only fitting.

In the next lesson, you will learn how to package EDA insights into a report that is actionable for both technical and non-technical stakeholders.